# Client

Demo of a client interacting with a remote agent. 

You can interact with this via API directly

In [1]:
import requests

inputs = {"input": {"input": "what does eugene think of cats?"}}
response = requests.post("http://localhost:8000/invoke", json=inputs)

response.json()

{'output': {'output': 'Eugene thinks that cats like fish.'},
 'metadata': {'run_id': '78a6d2c5-4982-406b-b356-0ca213086555',
  'feedback_tokens': []}}

You can also interact with this via the RemoteRunnable interface (to use in other chains)

In [2]:
from langserve import RemoteRunnable

remote_runnable = RemoteRunnable("http://localhost:8000/")

Remote runnable has the same interface as local runnables

In [3]:
await remote_runnable.ainvoke({"input": "hi!"})

{'output': 'Hello! How can I assist you today?'}

In [4]:
remote_runnable.invoke({"input": "what does eugene think of cats?"})

{'output': 'Eugene thinks that cats like fish.'}

## Stream

Please note that streaming alternates between actions and observations. It does not stream individual tokens! If you need to stream individual tokens you will need to use astream_log!

In [5]:
async for chunk in remote_runnable.astream({"input": "what does eugene think of cats? Then tell me a story about that thought."}):
    print('--')
    print(chunk)

--
{'actions': [AgentActionMessageLog(tool='get_eugene_thoughts', tool_input={'query': 'cats'}, log="\nInvoking: `get_eugene_thoughts` with `{'query': 'cats'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "cats"\n}', 'name': 'get_eugene_thoughts'}}, response_metadata={'finish_reason': 'function_call'}, id='run-0ee02e6e-d436-4a8d-9a6e-935c74a1bad8')])], 'messages': [AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "cats"\n}', 'name': 'get_eugene_thoughts'}}, response_metadata={'finish_reason': 'function_call'}, id='run-0ee02e6e-d436-4a8d-9a6e-935c74a1bad8')]}
--
{'steps': [{'action': AgentActionMessageLog(tool='get_eugene_thoughts', tool_input={'query': 'cats'}, log="\nInvoking: `get_eugene_thoughts` with `{'query': 'cats'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "cats"\n}', 'name': 'get_eugene_thoughts'}

## Stream Events

The client is looking for a runnable name called `agent` for the chain events. This name was defined on the server side using `runnable.with_config({"run_name": "agent"}`

In [6]:
async for event in remote_runnable.astream_events(
    {"input": "what does eugene think of cats? Then tell me a story about that thought."},
    version="v1",
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

Starting agent: agent with input: {'input': 'what does eugene think of cats? Then tell me a story about that thought.'}
--
Starting tool: get_eugene_thoughts with inputs: {'query': 'cats'}
Done tool: get_eugene_thoughts
Tool output was: [Document(page_content='cats like fish'), Document(page_content='dogs like sticks')]
--
E|ug|ene| thinks| that| cats| like| fish|.| Now|,| let| me| tell| you| a| story| about| Eugene|'s| thoughts| on| cats|.

|Once| upon| a| time|,| in| a| small| village|,| there| lived| a| mis|chie|vous| cat| named| Wh|isk|ers|.| Wh|isk|ers| was| known| for| his| love| of| fish|.| Every| day|,| he| would| sneak| into| the| village|'s| fish| market| and| try| to| snatch| a| fresh| fish| for| himself|.

|One| day|,| Eugene|,| a| curious| observer|,| noticed| Wh|isk|ers|'| behavior| and| became| fascinated| by| the| cat|'s| obsession| with| fish|.| Eugene| decided| to| follow| Wh|isk|ers| and| see| where| his| adventures| would| lead|.

|As| Eugene| followed| Wh|isk|ers|,

## Stream log

If you need acccess the individual llm tokens from an agent use `astream_log`. Please make sure that you set **streaming=True** on your LLM (see server code). For this to work, the LLM must also support streaming!

In [7]:
async for chunk in remote_runnable.astream_log({"input": "what does eugene think of cats?"}):
    print('--')
    print(chunk)

--
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '24c03cd4-f49e-4bb6-96c7-8f67bc7fa13b',
            'logs': {},
            'name': 'agent',
            'streamed_output': [],
            'type': 'chain'}})
--
RunLogPatch({'op': 'add',
  'path': '/logs/RunnableSequence',
  'value': {'end_time': None,
            'final_output': None,
            'id': 'fd41d343-b1a9-4f78-b4b7-a3a80f6de867',
            'metadata': {'__langserve_endpoint': 'stream_log',
                         '__langserve_version': '0.2.1',
                         '__useragent': 'python-httpx/0.27.0'},
            'name': 'RunnableSequence',
            'start_time': '2024-06-15T02:37:29.905+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': [],
            'type': 'chain'}})
--
RunLogPatch({'op': 'add',
  'path': '/logs/RunnableParallel<input,agent_scratchpad>',
  'value': {'end_time': None,
            'final_outpu